In [1]:
import os
os.environ['CONDA_DLL_SEARCH_MODIFICATION_ENABLE']="1"
import numpy as np
import pandas as pd
#import pyswmm.toolkitapi as tkai

from swmm_api.input_file import read_inp_file
from pyswmm import Simulation,Links,Nodes,RainGages,SystemStats
from swmm_api.input_file.sections.others import TimeseriesData
from swmm_api.input_file.sections import Control
from swmm_api.input_file.section_labels import TIMESERIES, CONTROLS
from swmm_api import read_rpt_file, SwmmReport

import matplotlib.pyplot as plt
import datetime
import yaml
import shutil 
import random
import math
from joblib import Parallel, delayed

e:\anaconda3\envs\tensorflow-cpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
def get_step_results(results,nodes,links,rgs,sys,config,params):
    
    #获取reward
    delt_flooding,delt_CSO,CSOtem,inflow=0,0,0,0
    for _temp in config['reward_targets']:
        if _temp[1] == 'flooding':
            if _temp[0] == 'system':
                delt_flooding += sys.routing_stats[_temp[1]] - results['flooding'][-1]
            else:
                delt_flooding += nodes[_temp[0]].statistics['flooding_volume']
            results['flooding'].append(sys.routing_stats[_temp[1]])
        else:
            #cum_cso = sys.routing_stats['outflow']
            CSOtem += nodes[_temp[0]].cumulative_inflow
            # log the cumulative value
            #self.data_log[_temp[1]][_temp[0]].append(cum_cso)
    delt_CSO = CSOtem - results['CSO'][-1]
    results['CSO'].append(CSOtem)
            
    Qtw = (sys.routing_stats['dry_weather_inflow']
                +sys.routing_stats['wet_weather_inflow']
                +sys.routing_stats['groundwater_inflow']
                +sys.routing_stats['II_inflow'])
    delt_Qtw = Qtw - results['inflow'][-1]
    results['inflow'].append(Qtw)
    
    #flooding time and cso time
    floodingt, delt_flooding_time = 0,0
    for n in nodes:
        if n.statistics['flooding_duration'] >= floodingt:
            floodingt = n.statistics['flooding_duration']
    delt_flooding_time = floodingt - results['total_flooding_time'][-1]
    results['total_flooding_time'].append(floodingt)

    delt_cso_time = 0
    if delt_CSO > 0:
        delt_cso_time = params['advance_seconds']/3600
    results['total_CSO_time'].append(results['total_CSO_time'][-1]+delt_cso_time)

    # 3 types of reward
    if params['reward_type'] == '1':
        if results['inflow'][-1] == 0:
            reward = 0
        else:
            reward = - (1/(results['inflow'][-1])) * ((delt_flooding * results['total_flooding_time'][-1] + delt_flooding_time * results['flooding'][-1]) 
                                        + (delt_CSO * results['total_CSO_time'][-1] + delt_cso_time * results['CSO'][-1]))
    else:
        if results['inflow'][-1] == 0:
            reward = 0
        else:
            reward = 1 / (1 + delt_flooding/Qtw + delt_CSO/Qtw) - 1
    return results, reward


In [3]:
# using OPT to find best traj for reward1
class SWMM_OPT:
    #can be used for every SWMM inp
    def __init__(self,params):
        '''
        params: a dictionary with input
        orf: original file of swmm inp
        control_asset: list of contorl objective, pumps' name
        advance_seconds: simulation time interval
        flood_nodes: selected node for flooding checking
        '''
        self.params = params
        self.config = yaml.load(open(self.params['parm']+".yaml"), yaml.FullLoader)
        #self.t=[]
        self.action_table = pd.read_csv(os.path.dirname(os.getcwd())+'/v1/SWMM/action_table.csv').values[:,1:]
    
    def reset(self,rain,rainid,root):
        
        root += '/SWMM/_temopt'
        inp = read_inp_file(self.params['orf']+'.inp')
        inp[TIMESERIES]['rainfall']=TimeseriesData('rainfall',rain)
        inp.write_file(root+'/'+self.params['orf_save']+str(rainid)+'_rain_opt.inp')
        self.sim=Simulation(root+'/'+self.params['orf_save']+str(rainid)+'_rain_opt.inp')

        # 获取结果，记录cso、flooding
        self.results = {}
        
        #记录总体cso, flooding, res
        self.results['CSO'], self.results['flooding'], self.results['inflow'] = [0], [0], [0]
        self.results['total_flooding_time'], self.results['total_CSO_time'] = [0], [0]
        self.results['res'] = [0]
        self.results['state'], self.results['action'], self.results['rewards'] = [], [], []
        
    
    def sim_onerain_v1(self,actions,rain,rainid,root,mpi):
        # 求解优化用的inp与最终模拟计算用的inp不同
        # 将控制变量转化为泵站控制指令，7台泵
        def action_op(actions):    
            delta,dura = 1, len(actions)
            act = {}
            for pumpid in range(7):
                ts = []
                for i in range(dura//delta):
                    t = i*delta
                    if np.mod(t,5)==0:
                        key = '08/28/2015 '+str(8+t//60).zfill(2)+':'+str(t % 60).zfill(2)+':'+'00'
                        ts.append([key,self.action_table[actions[i]][pumpid]])
                act['pump'+str(pumpid)] = ts
            return act

        root += '/SWMM/_temopt'
        inp = read_inp_file(self.params['orf']+'_opt.inp')
        inp[TIMESERIES]['rainfall']=TimeseriesData('rainfall',rain[rainid])
        at = action_op(actions)
        inp[TIMESERIES]['pump0']=TimeseriesData('pump0',at['pump0'])
        inp[TIMESERIES]['pump1']=TimeseriesData('pump1',at['pump1'])
        inp[TIMESERIES]['pump2']=TimeseriesData('pump2',at['pump2'])
        inp[TIMESERIES]['pump3']=TimeseriesData('pump3',at['pump3'])
        inp[TIMESERIES]['pump4']=TimeseriesData('pump4',at['pump4'])
        inp[TIMESERIES]['pump5']=TimeseriesData('pump5',at['pump5'])
        inp[TIMESERIES]['pump6']=TimeseriesData('pump6',at['pump6'])

        inp.write_file(root+'/'+self.params['orf_save']+'_rain'+str(rainid)+'_opt'+str(mpi)+'.inp')
        with Simulation(root+'/'+self.params['orf_save']+'_rain'+str(rainid)+'_opt'+str(mpi)+'.inp') as sim:
            #stand_reward=0
            for step in sim:
                pass

        # 读取结果，仅读取flooding与CSO之和作为优化标准
        # flooding
        rpt = read_rpt_file(root+'/'+self.params['orf_save']+'_rain'+str(rainid)+'_opt'+str(mpi)+'.rpt')  # type: swmm_api.SwmmReport
        node_flooding_summary = np.sum(rpt.node_flooding_summary['Total_Flood_Volume_10^6 ltr'])
        #CSO
        CSO = 0
        for csoi in [1,2,3,4]:
            CSO += rpt.node_inflow_summary['Total_Inflow_Volume_10^6 ltr'][self.config['reward_targets'][csoi][0]]
        #inflow
        inflow = rpt.flow_routing_continuity['Dry Weather Inflow']['Volume_10^6 ltr']+\
                rpt.flow_routing_continuity['Wet Weather Inflow']['Volume_10^6 ltr']+\
                rpt.flow_routing_continuity['Groundwater Inflow']['Volume_10^6 ltr']+\
                rpt.flow_routing_continuity['RDII Inflow']['Volume_10^6 ltr']+\
                rpt.flow_routing_continuity['External Inflow']['Volume_10^6 ltr']
        if inflow == 0:
            r = 0
        else:
            r = (-node_flooding_summary-CSO)/inflow
        return r


    def sim_onerain_v2(self,actions,rain,rainid,root):
        self.reset(rain[rainid],rainid,root)
        self.sim.start()
        
        for t in range(len(rain[rainid])):
            #模拟一步
            if self.params['advance_seconds'] is None:
                self.sim._model.swmm_step()
            else:
                self.sim._model.swmm_stride(self.params['advance_seconds'])
                    
            #obtain states and reward term by yaml (config)
            nodes = Nodes(self.sim)
            links = Links(self.sim)
            rgs = RainGages(self.sim)
            sys = SystemStats(self.sim)
            states = []
            for _temp in self.config["states"]:
                if _temp[1] == 'depthN':
                    states.append(nodes[_temp[0]].depth)
                elif _temp[1] == 'flow':
                    states.append(links[_temp[0]].flow)
                elif _temp[1] == 'inflow':
                    states.append(nodes[_temp[0]].total_inflow)
                else:
                    states.append(rgs[_temp[0]].rainfall)
            
            #获取reward和结果
            self.results, reward = get_step_results(self.results,nodes,links,rgs,sys,self.config,self.params)

            self.results['state'].append(states)
            self.results['action'].append(actions[t])
            self.results['rewards'].append(reward)

        return self.results

In [7]:
action_table = pd.read_csv(os.path.dirname(os.getcwd())+'/v1/SWMM/action_table.csv').values[:,1:]
raindata = np.load(os.path.dirname(os.getcwd())+'/v1/rainfall/training_raindata.npy').tolist()

In [9]:
# Single test
actions = [np.random.randint(0,128) for _ in range(len(raindata[0]))]

env_params={
        'orf':os.path.dirname(os.getcwd())+'/v1/SWMM/chaohu',
        'orf_save':'chaohu_OPT',
        'parm':os.path.dirname(os.getcwd())+'/v1/states_yaml/chaohu',
        'advance_seconds':300,
        'kf':1,
        'kc':1,
        'reward_type':'2'
    }
env=SWMM_OPT(env_params)
#results = env.sim_onerain_v2(actions,raindata,0,os.path.dirname(os.getcwd())+'/v1')

In [5]:
def calobjValue(actions_set, raindata, raindataid):
    env = SWMM_OPT(env_params)
    obj_value=[]
    for actions in actions_set:
        #results = env.sim_onerain_v2(actions,raindata,raindataid,os.path.dirname(os.getcwd())+'/v1')
        # 根据flooding+CSO计算得分
        #obj_value.append(np.mean(results['rewards']))
        results = env.sim_onerain_v1(actions,raindata,raindataid,os.path.dirname(os.getcwd())+'/v1')
        # 根据flooding+CSO计算得分
        obj_value.append(np.mean(results))
    return obj_value

def calobjValue_mp(actions_set, raindata, raindataid):
    
    def interact(actions,raindata,raindataid,root,mpi):
        env = SWMM_OPT(env_params)
        results = env.sim_onerain_v1(actions,raindata,raindataid,root,mpi)
        return results
    obj_value = Parallel(n_jobs=10)(delayed(interact)(actions_set[i],raindata,raindataid,os.path.dirname(os.getcwd())+'/v1',i) for i in range(len(actions_set))) 
    return obj_value

def get_action(pop, max_value):
    # pop是pop_size个，每个chrom_length长的2进制数，转化为1到128的实数
    actions_set = []
    for i in range(len(pop)):
        actions=[]
        for t in range(len(pop[i])):
            actions.append(int(max_value*pop[i][t]))
        actions_set.append(actions)
    return actions_set

def opt(rain,rainid,pop_size,chrom_length,max_value,pc,pm,optstep):

    pop = geneEncoding(pop_size, chrom_length)
    results = [[]]      # 存储每一代的最优解，N个二元组
    fit_value = []      # 个体适应度
    fit_mean = []       # 平均适应度
    for i in range(optstep):
        if np.mod(i,1000)==0:
            print('step:',i)
        
        actions_set = get_action(pop, max_value)        # 个体评价
        fit_value = calobjValue_mp(actions_set,rain,rainid)       # 淘汰
        best_individual, best_fit = best(pop, fit_value)		# 第一个存储最优的解, 第二个存储最优基因
        results.append([best_fit, best_individual])
        selection(pop, fit_value)		# 新种群复制
        crossover(pop, pc)		# 交配
        mutation(pop, pm)       # 变异

    best_individual, best_fit = best(pop, fit_value)
    return best_individual, best_fit

# 开始寻优

In [ ]:
raindataid = 2
best_individual, best_fit = opt(raindata,raindataid,10,len(raindata[0]),127,0.6,0.5,3000)
print(best_fit)

In [ ]:
rpt = read_rpt_file('pump.rpt')  # type: swmm_api.SwmmReport
node_flooding_summary = np.sum(rpt.node_flooding_summary['Total_Flood_Volume_10^6 ltr'])
CSO = 0
for csoi in [1,2,3,4]:
    CSO += rpt.node_inflow_summary['Total_Inflow_Volume_10^6 ltr'][env.config['reward_targets'][csoi][0]]
#inflow
inflow = rpt.flow_routing_continuity['Dry Weather Inflow']['Volume_10^6 ltr']+\
        rpt.flow_routing_continuity['Wet Weather Inflow']['Volume_10^6 ltr']+\
        rpt.flow_routing_continuity['Groundwater Inflow']['Volume_10^6 ltr']+\
        rpt.flow_routing_continuity['RDII Inflow']['Volume_10^6 ltr']+\
        rpt.flow_routing_continuity['External Inflow']['Volume_10^6 ltr']
if inflow == 0:
    r = 0
else:
    r = (-node_flooding_summary-CSO)/inflow
print(r)

# 将找到control模拟一次并保存，由于后续DRL训练的reward1

In [ ]:
env = SWMM_OPT(env_params)
root = os.path.dirname(os.getcwd())+'/v1'
actions_set = get_action([best_individual], 127)
results = env.sim_onerain_v2(actions,raindata,raindataid,root)
plt.plot(results['rewards'])

np.save('./OPT_results/opt_results_rain'+str(raindataid)+'.npy',results)
# rrr = np.load('./OPT_results/opt_results_rain'+str(raindataid)+'.npy',allow_pickle=True).tolist()

# 自动迭代上面两步

In [10]:
root = os.path.dirname(os.getcwd())+'/v1'
for raindataid in range(45,50):
    best_individual, best_fit = opt(raindata,raindataid,10,len(raindata[0]),127,0.6,0.5,3000)
    print(raindataid,': ',best_fit)
    env = SWMM_OPT(env_params)
    
    actions_set = get_action([best_individual], 127)
    results = env.sim_onerain_v2(actions,raindata,raindataid,root)
    #plt.plot(results['rewards'])
    np.save('./OPT_results/opt_results_rain'+str(raindataid)+'.npy',results)
    # rrr = np.load('./OPT_results/opt_results_rain'+str(raindataid)+'.npy',allow_pickle=True).tolist()

step: 0
step: 1000
step: 2000
45 :  -0.538920371560437
step: 0
step: 1000
step: 2000
46 :  -0.616927746572236
step: 0
step: 1000
step: 2000
47 :  -0.5702526310064449
step: 0
step: 1000
step: 2000
48 :  -0.501982763178766
step: 0
step: 1000
step: 2000
49 :  -0.6778069846038302
